In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
deep_reload(src)

n = 1000
scale = (1e6, 1e6)
reference_distance = 200e3
seed = 129687083

graph = src.rng.random_graph(
    n,
    scale = scale,
    reference_distance = reference_distance,
    seed = seed,
)

origins = src.rng.random_nodes(graph, 1)
destinations = src.rng.random_nodes(graph, 10)
stations = src.rng.random_nodes(graph, 15)

In [3]:
deep_reload(src)

t0 = time.time()

path_costs, path_values, paths = src.routing.dijkstra(graph, origins)

print(time.time() - t0)

0.13073325157165527


In [4]:
deep_reload(src)

objective = src.routing.Vehicle()

t0 = time.time()

path_costs, path_values, paths = src.routing.dijkstra(
    graph, origins, objective = objective)

print(time.time() - t0)

KeyError: 'type'

In [ ]:
len(path_costs)

In [ ]:
t0 = time.time()

r = nx.single_source_dijkstra(graph, origins[0], weight = 'time')
print(time.time() - t0)

In [5]:
t0 = time.time()

r = nx.single_source_bellman_ford(graph, origins[0], weight = 'time')
print(time.time() - t0)

0.13633179664611816


In [89]:
graph = src.graph.graph_from_json('Outputs/graph_full.json')

In [96]:
for source, adj in graph._adj.items():
    for target, edge in adj.items():
        
        graph._node[source]['time'] = 0
        graph._node[source]['distance'] = 0
        graph._node[source]['price'] = 0

        graph._node[target]['time'] = 0
        graph._node[target]['distance'] = 0
        graph._node[target]['price'] = 0

        edge['price'] = 0

        if ('station' in source):

            graph._node[source]['type'] = 'station'

        else:

            graph._node[source]['type'] = 'city'

        if ('station' in source) and ('station' in target):
            
            edge['type'] = 'station_station'

        elif ('station' in source) or ('station' in target):

            edge['type'] = 'city_station'
            
        else:
            
            edge['type'] = 'city_city'

In [97]:
deep_reload(src)

objective = src.dijkstra.Objective(field = 'time')

t0 = time.time()

r = src.dijkstra.dijkstra(graph, ['Fresno'], objective = objective)

print(time.time() - t0)

2.544259786605835


In [98]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.6082676698301681,
 'Stockton': 1.9591595904095878,
 'San Jose': 2.5138573648573628,
 'Sacramento': 2.6713354700854657,
 'San Francisco': 2.9443037864912807,
 'Los Angeles': 3.263417207792204,
 'Chico': 4.230472506660002,
 'I-80 to Reno (State Line)': 4.53776988636363,
 'Redding': 5.118724469974463,
 'San Diego': 5.252874417249402,
 'I-15 to Las Vegas (State Line)': 5.727073083166826,
 'Yreka': 6.610469062187799,
 'I-10 to Phoenix (State Line)': 6.647205294705284,
 'Crescent City': 8.605738955488926}

In [99]:
deep_reload(src)

objective = src.routing.Vehicle(capacity = 80 * 3.6e6)

t0 = time.time()

r = src.dijkstra.dijkstra(graph, ['Fresno'], objective = objective, return_paths = True)

print(time.time() - t0)

13.758971929550171


In [100]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.6082676698301694,
 'Stockton': 1.9591595904095913,
 'San Jose': 2.513857364857367,
 'Sacramento': 2.6713354700854763,
 'San Francisco': 2.9443037864912918,
 'Los Angeles': 3.263417207792208,
 'Chico': 4.23047250666002,
 'I-80 to Reno (State Line)': 4.5377698863636535,
 'Redding': 5.228450164418934,
 'San Diego': 5.380952542249436,
 'I-15 to Las Vegas (State Line)': 5.916281416500174,
 'Yreka': 7.018837117743368,
 'I-10 to Phoenix (State Line)': 7.146036891927523,
 'Crescent City': 9.353999372155622}

In [187]:
deep_reload(src)

objective = src.routing.Vehicle(capacity = 80 * 3.6e6)

t0 = time.time()

r = src.bellman.bellman(
    graph, ['Fresno'], objective = objective, return_paths = True
)

print(time.time() - t0)

3.746652364730835


In [188]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Chico': 4.23047250666002,
 'I-80 to Reno (State Line)': 4.5377698863636535,
 'Sacramento': 2.6713354700854763,
 'Stockton': 1.9591595904095913,
 'San Francisco': 2.9443037864912918,
 'San Jose': 2.513857364857367,
 'Bakersfield': 1.6082676698301694,
 'Los Angeles': 3.263417207792208,
 'Crescent City': 9.353999372155622,
 'Yreka': 7.018837117743368,
 'Redding': 5.228450164418934,
 'I-15 to Las Vegas (State Line)': 5.916281416500174,
 'I-10 to Phoenix (State Line)': 7.146036891927523,
 'San Diego': 5.380952542249436}

In [189]:
{k: v['driving_time'] / 3600 for k, v in r[1].items() if 'station' not in k}

{'Fresno': 0.0,
 'Chico': 4.23047250666002,
 'I-80 to Reno (State Line)': 4.5377698863636535,
 'Sacramento': 2.6713354700854763,
 'Stockton': 1.9591595904095913,
 'San Francisco': 2.9443037864912918,
 'San Jose': 2.513857364857367,
 'Bakersfield': 1.6082676698301694,
 'Los Angeles': 3.263417207792208,
 'Crescent City': 8.605738955488954,
 'Yreka': 6.610469062187812,
 'Redding': 5.118724469974489,
 'I-15 to Las Vegas (State Line)': 5.7270730831668395,
 'I-10 to Phoenix (State Line)': 6.647205294705299,
 'San Diego': 5.252874417249437}

In [190]:
r[2]['Crescent City']

['Fresno', 'station_132', 'Crescent City']

In [191]:
r[1]['station_301'], 19.685 / .5 / 80, objective.range / 1e3

({'time': 11061.813124375623,
  'driving_time': 8990.125624375623,
  'distance': 257700.0,
  'price': 19.685416666666665,
  'soc': 1.0},
 0.492125,
 523.6363636363636)

In [192]:
r[1]['Crescent City']

{'time': 33674.397739760236,
 'driving_time': 30980.660239760233,
 'distance': 857710.0,
 'price': 26.597083333333327,
 'soc': 0.026939236111111153}

In [193]:
deep_reload(src)

objective = src.routing.Vehicle(
    capacity = 528 * 3.6e6,
    efficiency = 550 * 5,
    charge_rate = 14.15e6,
    charge_time_penalty = 300,
)

t0 = time.time()

r = src.bellman.bellman(
    graph, ['Fresno'], objective = objective, return_paths = True
)

print(time.time() - t0)

4.632653474807739


In [194]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Redding': 5.11872446997449,
 'Chico': 4.23047250666002,
 'I-80 to Reno (State Line)': 4.5377698863636535,
 'Sacramento': 2.6713354700854763,
 'Stockton': 1.9591595904095913,
 'San Francisco': 2.9443037864912918,
 'San Jose': 2.513857364857367,
 'I-15 to Las Vegas (State Line)': 5.727073083166839,
 'Bakersfield': 1.6082676698301694,
 'Los Angeles': 3.263417207792208,
 'San Diego': 5.252874417249402,
 'Yreka': 6.628059943528701,
 'Crescent City': 8.69826054952113,
 'I-10 to Phoenix (State Line)': 6.731803498474445}

In [195]:
{k: v['driving_time'] / 3600 for k, v in r[1].items() if 'station' not in k}

{'Fresno': 0.0,
 'Redding': 5.11872446997449,
 'Chico': 4.23047250666002,
 'I-80 to Reno (State Line)': 4.5377698863636535,
 'Sacramento': 2.6713354700854763,
 'Stockton': 1.9591595904095913,
 'San Francisco': 2.9443037864912918,
 'San Jose': 2.513857364857367,
 'I-15 to Las Vegas (State Line)': 5.727073083166839,
 'Bakersfield': 1.6082676698301694,
 'Los Angeles': 3.263417207792208,
 'San Diego': 5.252874417249402,
 'Yreka': 6.628059943528701,
 'Crescent City': 8.605738955488937,
 'I-10 to Phoenix (State Line)': 6.647205294705305}

In [196]:
r[2]['Crescent City']

['Fresno', 'station_354', 'Crescent City']

In [197]:
r[1]['Crescent City']

{'time': 31313.73797827607,
 'driving_time': 30980.66023976017,
 'distance': 857710.0,
 'price': 65.00694444444444,
 'soc': 0.005338541666666696}

In [131]:
objective.range / 1e3

691.2

In [67]:
deep_reload(src)

objective = src.bellman.Objective(field = 'time')

t0 = time.time()

r = src.bellman.bellman(graph, ['Fresno'], objective = objective, return_paths = True)

print(time.time() - t0)

1.0945961475372314


In [68]:
{k: v / 3600 for k, v in r[0].items() if 'station' not in k}

{'Fresno': 0.0,
 'Sacramento': 2.7117650058275027,
 'Stockton': 2.0049909396159387,
 'San Francisco': 2.9574284049284016,
 'San Jose': 2.5324649794649785,
 'Bakersfield': 1.608267669830169,
 'Los Angeles': 3.2643437812187788,
 'Redding': 5.141404012654007,
 'Chico': 4.307111145798643,
 'I-80 to Reno (State Line)': 4.586502074314568,
 'I-15 to Las Vegas (State Line)': 5.820441589660336,
 'San Diego': 5.253800990675983,
 'Yreka': 6.627662493756237,
 'I-10 to Phoenix (State Line)': 6.672202103452094,
 'Crescent City': 8.611122946497934}

In [70]:
r[2]['Crescent City']

['Fresno',
 'station_1371',
 'station_724',
 'station_1063',
 'station_1131',
 'station_1800',
 'station_420',
 'station_1192',
 'station_342',
 'station_723',
 'station_513',
 'station_1145',
 'station_1634',
 'Crescent City']

In [310]:
objective.range / 1.609e3

325.4421153737499